# Agent Supervisor Architecture Application
Applying a supervisor approach to ideas of social media sentiment analysis

In [138]:
import os
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

---

## Gathering API keys

LLMs and LangChain API key set in environment variable, `langchain_tracing_v2` allows langsmith agent tracing, `langchain_project` determines organization in langsmith

In [139]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "Experiments Master"
# os.environ["OPENAI_API_KEY"] = "apikey"

In [140]:
llm = Ollama(model="llama3")

In [141]:
llm.invoke("how can langsmith help with testing?")

"Languagem is a fantastic tool for language learners, and it can also be quite helpful in testing. Here are some ways Langsmith can assist:\n\n1. **Error detection**: Langsmith's AI-powered algorithms can analyze text written by learners and identify grammatical errors, inaccuracies, or inconsistencies. This helps teachers and tutors to pinpoint areas where students need improvement.\n2. **Grammar and syntax checks**: Langsmith's grammar rules and linguistic patterns can be used to test whether learner-generated texts conform to the target language's rules and conventions. This is particularly useful for testing sentence structure, verb tenses, and other grammatical aspects.\n3. **Vocabulary assessment**: Langsmith can help evaluate learners' vocabulary by analyzing their text-based responses and identifying unfamiliar or misused words. This feedback enables teachers to focus on teaching specific vocabulary items.\n4. **Readability and comprehension checks**: Langsmith's readability me

In [134]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation reader."),
    ("user", "{input}")
])
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "What is the Add to 'Dataset functionality' in LangSmith?"})

'As a world-class technical documentation reader, I\'m delighted to dive into the intricacies of Langsmith\'s \'Add to Dataset\' functionality.\n\nFrom what I\'ve gathered from Langsmith\'s official documentation and user feedback, the \'Add to Dataset\' feature is an essential component of their data annotation tool. Here\'s how it works:\n\nWhen you\'re working on a project in Langsmith, you might encounter a situation where you need to add new information or modify existing data to improve the accuracy and quality of your dataset. That\'s where \'Add to Dataset\' comes into play.\n\nThis feature allows you to seamlessly integrate new data into an existing dataset, ensuring that all relevant information is preserved and organized in a single location. You can think of it as a "dataset aggregator" or a "data merger."\n\nTo utilize this functionality, follow these general steps:\n\n1. Open the project containing the dataset you want to update.\n2. Navigate to the specific dataset file 

In [135]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

embeddings = OllamaEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'According to the context, Langsmith can help with testing by allowing you to "visualize test results".'

In [136]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...
# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

Based on the provided context, LangSmith can help with testing in the following ways:

1. **Initial Test Set**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
2. **Comparison View**: Langsmith provides a user-friendly comparison view for test runs, allowing users to track and diagnose regressions in test scores across multiple revisions of their application.
3. **Logging**: Every playground run is logged in the system, which can be used to create test cases or compare with other runs.
4. **Automations**: Automations are a powerful feature that allow you to perform actions on traces in near real-time, such as automatically scoring traces, sending them to annotation queues, or sending them to datasets.
5. **Threads**: LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is fa

In [137]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fi